# Inserción de Datos - Jóvenes a la E

Este notebook se encarga de la carga y procesamiento de datos para poblar las tablas de la base de datos del programa Jóvenes a la E.

## Contenido

1. **Configuración Inicial**
   - Importación de librerías necesarias (pandas, mysql.connector)
   - Establecimiento de conexión con la base de datos

2. **Fuente de Datos**
   - Lectura del archivo Excel 'je_conteo_20241231.xlsx'
   - Procesamiento inicial de datos

3. **Procesamiento por Tabla**

   ### Localidad
   - Extracción de datos de localidades
   - Limpieza y normalización de nombres
   - Inserción en tabla localidad

   ### Institución Educativa Superior
   - Procesamiento de códigos SNIES
   - Normalización de nombres de instituciones
   - Inserción en tabla inst_edu_superior

   ### Programa
   - Extracción de información de programas académicos
   - Procesamiento de modalidades
   - Vinculación con instituciones educativas
   - Inserción en tabla programa

   ### Estudiantes
   - Procesamiento de datos demográficos
   - Normalización de categorías
   - Validación de datos
   - Inserción en tabla estudiante

   ### Convocatoria
   - Procesamiento de información de convocatorias
   - Extracción de año y semestre
   - Inserción en tabla convocatoria

   ### Participación
   - Vinculación de estudiantes con programas y convocatorias
   - Validación de integridad referencial
   - Inserción en tabla participacion

## Notas Técnicas
- Uso de pandas para manipulación de datos
- Implementación de validaciones de datos
- Manejo de excepciones para errores de inserción
- Procesamiento por lotes para optimizar rendimiento

In [1]:
# Importar librerías
import pandas as pd 
import mysql.connector as mysql
import random
from pathlib import Path
import logging
import sys 

In [2]:
# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

# Conexión a la base de datos
database_connection = mysql.connect(
    host ='localhost',
    port = 3306,
    user = 'root',
    password = 'adso00',
    database = 'jovenes_a_la_e'
)

if database_connection.is_connected():
    logging.info("Conexión exitosa a la base de datos")


2025-06-02 11:32:02,120 - INFO - Conexión exitosa a la base de datos


In [3]:
# Extracción de datos
df = pd.read_excel(r'C:\Users\andre\OneDrive\Documentos\jovenes_a_la_e\data\je_conteo_20241231.xlsx')

# Extracción de columnas relevantes para las tablas creadas

# Tabla de Localidad
df_localidad = df[[
    'LOCALIDAD'
]].copy()

# Tabla de Institución Educativa Superior
df_institucion = df[[
    'CODIGO SNIES IES',
    'NOMBRE INSTITUCION EDUCACION SUPERIOR'
]].copy()

# Tabla de Programa
df_programa = df[[
    'NUCLEO BASICO DEL CONOCIMIENTO PROGRAMA',
    'MODALIDADCORTE',
    'CODIGO SNIES IES'
]].copy()

# Tabla de Estudiantes
df_estudiantes = df[[
    'EDAD AL MOMENTO DE LA CONVOCATORIA',
    'SEXO',
    'GRUPO ETNICO',
    'VICTIMA DEL CONFLICTO ARMADO',
    'DISCAPACIDAD',
    'GRUPOS SISBEN 4',
    'SABER11 - PERCENTIL GLOBAL',
    'ZONA COLEGIO GRADUACION MEDIA',
    'SECTOR COLEGIO GRADUACION MEDIA'
]].copy()

# Tabla de Convocatoria
df_convocatoria = df[[
    'CONVOCATORIA',
]].copy()

In [4]:
# Renombrar columnas para que coincidan con la tabla SQL
df_localidad.columns = [
    'nombre_localidad'
]

df_institucion.columns = [
    'inst_edu_superior_id',
    'nombre_inst_edu_superior'
]

df_programa.columns = [
    'nombre_programa',
    'modalidad',
    'inst_edu_superior_id'
]

df_estudiantes.columns = [
    'rango_edad',
    'genero',
    'grupo_etnico',
    'victima_conflicto_arm',
    'discapacidad',
    'grupo_sisben',
    'percentil_saber11',
    'zona_colegio',
    'sector_colegio'
]

df_convocatoria.columns = [
    'convocatoria_id'
]


In [5]:
# Validación de datos
# Verificar si hay datos duplicados y nulos en cada DataFrame 

def validar_y_reportar(df, nombre):
    """
    Reporta cantidad de nulos, duplicados y muestra info del DataFrame.
    """
    logging.info(f'Datos nan {nombre}:\n{df.isna().sum()}')
    logging.info(f'Datos null {nombre}:\n{df.isnull().sum()}')
    logging.info(f'Duplicados {nombre}:\n{df.duplicated().sum()}')
    df.info()

# Aplicar la función de validación a cada DataFrame
validar_y_reportar(df_estudiantes, "estudiantes")
validar_y_reportar(df_localidad, "localidad")
validar_y_reportar(df_institucion, "institucion")
validar_y_reportar(df_programa, "programa")
validar_y_reportar(df_convocatoria, "convocatoria")

2025-06-02 11:32:05,792 - INFO - Datos nan estudiantes:
rango_edad               0
genero                   0
grupo_etnico             0
victima_conflicto_arm    0
discapacidad             0
grupo_sisben             0
percentil_saber11        0
zona_colegio             0
sector_colegio           0
dtype: int64


2025-06-02 11:32:05,807 - INFO - Datos null estudiantes:
rango_edad               0
genero                   0
grupo_etnico             0
victima_conflicto_arm    0
discapacidad             0
grupo_sisben             0
percentil_saber11        0
zona_colegio             0
sector_colegio           0
dtype: int64
2025-06-02 11:32:05,834 - INFO - Duplicados estudiantes:
34708
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36963 entries, 0 to 36962
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   rango_edad             36963 non-null  object
 1   genero                 36963 non-null  object
 2   grupo_etnico           36963 non-null  object
 3   victima_conflicto_arm  36963 non-null  object
 4   discapacidad           36963 non-null  object
 5   grupo_sisben           36963 non-null  object
 6   percentil_saber11      36963 non-null  object
 7   zona_colegio           36963 non-null  object
 8   s

In [6]:
# Limpieza de datos
# Esta sección se encarga de limpiar los DataFrames eliminando filas duplicadas y filas que contengan valores nulos (`NaN`). 

def limpiar_y_reportar(df, nombre):
    """
    Elimina duplicados y nulos de un DataFrame, muestra info y retorna el DataFrame limpio.
    """
    df = df.drop_duplicates()
    df = df.dropna()
    logging.info(f"\nDataFrame {nombre} después de la limpieza:")
    df.info()
    return df

# Aplicar limpieza a cada DataFrame
df_localidad = limpiar_y_reportar(df_localidad, "localidad")
df_institucion = limpiar_y_reportar(df_institucion, "institucion")
df_programa = limpiar_y_reportar(df_programa, "programa")
df_convocatoria = limpiar_y_reportar(df_convocatoria, "convocatoria")

2025-06-02 11:32:05,923 - INFO - 
DataFrame localidad después de la limpieza:
<class 'pandas.core.frame.DataFrame'>
Index: 22 entries, 0 to 6559
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   nombre_localidad  22 non-null     object
dtypes: object(1)
memory usage: 352.0+ bytes
2025-06-02 11:32:05,939 - INFO - 
DataFrame institucion después de la limpieza:
<class 'pandas.core.frame.DataFrame'>
Index: 52 entries, 0 to 16113
Data columns (total 2 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   inst_edu_superior_id      52 non-null     int64 
 1   nombre_inst_edu_superior  52 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.2+ KB
2025-06-02 11:32:05,961 - INFO - 
DataFrame programa después de la limpieza:
<class 'pandas.core.frame.DataFrame'>
Index: 638 entries, 0 to 28590
Data columns (total 3 columns):
 #   Column     

In [7]:
# Poblar la base de datos
# Esta sección contiene el código para insertar los datos limpios de los DataFrames en las tablas correspondientes de la base de datos.

# Crear un cursor para ejecutar consultas
cursor = database_connection.cursor()

# 1. Localidades
localidad_ids = {}
for i, row in df_localidad.iterrows():
    cursor.execute("INSERT INTO localidad (nombre_localidad) VALUES (%s)", (row['nombre_localidad'],))
    database_connection.commit()
    localidad_ids[row['nombre_localidad']] = cursor.lastrowid # Guardar el ID de la localidad
    

# 2. Instituciones
for _, row in df_institucion.iterrows():
    cursor.execute("INSERT IGNORE INTO inst_edu_superior (inst_edu_superior_id, nombre_inst_edu_superior) VALUES (%s, %s)", 
                   (int(row['inst_edu_superior_id']), row['nombre_inst_edu_superior']))
    database_connection.commit()

# 3. Programas
for _, row in df_programa.iterrows():
    cursor.execute("INSERT INTO programa (nombre_programa, modalidad, inst_edu_superior_id) VALUES (%s, %s, %s)", 
                   (row['nombre_programa'], row['modalidad'], int(row['inst_edu_superior_id'])))
    database_connection.commit()

# 4. Convocatoria
convocatoria_info = {
    'JE1': (2024, 1),
    'JU6': (2023, 2),
    'JU5': (2023, 1),
    'JU4': (2022, 2),
    'JU3': (2022, 1),
    'JU2': (2021, 2),
    'JU1': (2021, 1)
}

for _, row in df_convocatoria.iterrows():
    anio, semestre = convocatoria_info.get(row['convocatoria_id'], (2020, 1))
    cursor.execute("""
        INSERT INTO convocatoria (convocatoria_id, anio, semestre)
        VALUES (%s, %s, %s)
    """, (row['convocatoria_id'], anio, semestre))
    database_connection.commit()

# 5. Estudiantes
df_estudiantes['localidad'] = df['LOCALIDAD']

# Mapear localidad a su ID
df_estudiantes['localidad_id'] = df_estudiantes['localidad'].map(localidad_ids)

# Verificar que no haya valores nulos en el mapeo
df_estudiantes = df_estudiantes.dropna(subset=['localidad_id'])
df_estudiantes['localidad_id'] = df_estudiantes['localidad_id'].astype(int)

# Documentos de identidad utilizados
usados = set()

for _, row in df_estudiantes.iterrows():
    # Generar documento único
    while True:
        doc = random.randint(1000000000, 9999999999)
        if doc not in usados:
            usados.add(doc)
            break
    # Obtener el ID de localidad
    localidad_id = row['localidad_id']
    valores = (
        str(doc), row['rango_edad'], row['genero'], row['grupo_etnico'],
        row['victima_conflicto_arm'], row['discapacidad'], row['grupo_sisben'],
        row['percentil_saber11'], row['zona_colegio'], row['sector_colegio'], localidad_id
    )
    # Insertar en la tabla estudiante
    cursor.execute("""
        INSERT INTO estudiante (
            documento_identidad, rango_edad, genero, grupo_etnico, victima_conflicto_arm, discapacidad,
            grupo_sisben, percentil_saber11, zona_colegio, sector_colegio, localidad_id
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, valores)
    database_connection.commit()


# 6. Participación
cursor.execute("SELECT estudiante_id FROM estudiante")
estudiantes_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT programa_id FROM programa")
programas_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT convocatoria_id FROM convocatoria")
convocatorias_ids = [row[0] for row in cursor.fetchall()]

# Crear una participación por estudiante con asignaciones aleatorias
for estudiante_id in estudiantes_ids:
    programa_id = random.choice(programas_ids)
    convocatoria_id = random.choice(convocatorias_ids)
    cursor.execute("""
        INSERT INTO participacion (estudiante_id, convocatoria_id, programa_id)
        VALUES (%s, %s, %s)
    """, (estudiante_id, convocatoria_id, programa_id))
    database_connection.commit()

# Confirmar la inserción de datos
logging.info("Datos insertados correctamente en la base de datos.")

2025-06-02 11:39:11,472 - INFO - Datos insertados correctamente en la base de datos.


In [8]:
# Validación de la cantidad de datos insertados

def reportar_cantidad(cursor, tabla, descripcion):
    cursor.execute(f"SELECT COUNT(*) FROM {tabla}")
    cantidad = cursor.fetchone()[0]
    logging.info(f"Total de {descripcion} insertados: {cantidad}")

# Aplicar la función de reporte a cada tabla
reportar_cantidad(cursor, "estudiante", "estudiantes")
reportar_cantidad(cursor, "localidad", "localidades")
reportar_cantidad(cursor, "inst_edu_superior", "instituciones educativas")
reportar_cantidad(cursor, "programa", "programas")
reportar_cantidad(cursor, "convocatoria", "convocatorias")
reportar_cantidad(cursor, "participacion", "participaciones")

# Cerrar el cursor y la conexión
cursor.close()
database_connection.close()

2025-06-02 11:39:11,522 - INFO - Total de estudiantes insertados: 36962
2025-06-02 11:39:11,537 - INFO - Total de localidades insertados: 22
2025-06-02 11:39:11,540 - INFO - Total de instituciones educativas insertados: 52
2025-06-02 11:39:11,549 - INFO - Total de programas insertados: 638
2025-06-02 11:39:11,549 - INFO - Total de convocatorias insertados: 7
2025-06-02 11:39:11,565 - INFO - Total de participaciones insertados: 36962
